In [1]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

Note: you may need to restart the kernel to use updated packages.


In [26]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [27]:
document_list[52]

Document(metadata={'source': './tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소

In [28]:
import os

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
upstage_api_key = os.environ.get("UPSTAGE_API_KEY")
embeddings = UpstageEmbeddings(
    api_key=upstage_api_key,
    model="embedding-query"
)

In [29]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-table-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
print(pinecone_api_key)

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

pcsk_DQeuF_Aqh9oSXDfiUJk1GXc3yYhzmm3inJPNAvad1qwKhq1tfw5v3Dcwk8rL78hJLdx1H


In [101]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

In [62]:
retriever = database.as_retriever(search_kwargs={'k': 2})
retriever.invoke(query)

[Document(id='e6a0e9c8-20c0-49f3-b0c8-b204e67b7535', metadata={'source': './tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2

In [63]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\tideflo\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [64]:
import os
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage

load_dotenv()
upstage_api_key = os.environ.get("UPSTAGE_API_KEY")

llm = ChatUpstage(
        api_key=upstage_api_key,
        model="solar-pro2"
)

In [65]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [105]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
                                          
    질문: {{question}}
""")

chain = prompt | llm | StrOutputParser()

In [106]:
new_question = chain.invoke({
    "question": query
})

In [89]:
query

'연봉 5천만원인 직장소득자의 종합소득세는?'

In [107]:
new_question

'사용자의 질문에서 이미 "사람"을 나타내는 표현으로 "거주자"가 사용되었습니다. 따라서 변경할 필요가 없습니다.  \n\n**답변:**  \n연봉 5천만원인 거주자의 종합소득세는?'

In [98]:
tax_chain = {"query": chain} | qa_chain

In [103]:
ai_response = tax_chain.invoke({"question": query})

In [104]:
ai_response

{'query': '사용자의 질문은 이미 사전에 제시된 변경 규칙("사람을 나타내는 표현 -> 거주자")을 반영한 상태입니다. "거주자"라는 표현이 사용되었으므로 추가 변경이 필요하지 않습니다.  \n\n**최종 질문**:  \n"연봉 5천만원인 거주자의 종합소득세는?"  \n\n(변경 없음)',
 'result': '연봉 5천만원인 거주자의 종합소득세는 624만원 + (5,000만원 - 1,400만원 = 3,600만원) × 15%로 계산되며, 총 1,164만원입니다. (단, 공제 및 추가 세액감면 미적용 시)  \n\n(계산: 624만원 + 3,600만원 × 0.15 = 1,164만원)'}

In [80]:
ai_message = qa_chain.invoke({"query": query})

In [81]:
ai_message['result']

'연봉 5천만원의 직장소득자의 종합소득세는 과세표준 구간 "5,000만원 초과 8,800만원 이하"에 해당하며, 세액은 **624만원 + (5,000만원 초과 금액의 24%)**로 계산됩니다. 단, 실제 세액 공제 및 추가 공제 적용 여부에 따라 최종 세액은 달라질 수 있습니다. 정확한 금액은 과세표준과 공제 항목을 확인한 후 계산해야 합니다.'